In [29]:
import numpy as np
import pandas as pd
import json
import re
import os
import ast
import matplotlib.pyplot as plt
from pymatgen.core.composition import Composition
from pymatgen.core.periodic_table import Species, Element
from disorder.composition import merged_comp
from disorder.composition import composition_from_formula

In [6]:
structure=pd.read_csv('data/ICSD-info.csv')

In [9]:
df=pd.read_json('data/all_extracted_data_no_H.json',orient='columns')

In [10]:
disorder=df.merge(structure,how='inner',left_on='ICSD_ID',right_on='Col_code')
disorder=disorder.loc[disorder['Temperature']<310]
disorder=disorder.loc[disorder['Temperature']>270]
disorder=disorder.loc[disorder['Pressure']<0.11]
disorder.reset_index(drop=True,inplace=True)

In [16]:
disorder=disorder.loc[~disorder['orbit_disorder'].isna()]
disorder.reset_index(drop=True,inplace=True)

In [17]:
len(disorder)

141654

In [18]:
disorder

,formula,ICSD_ID,group_num,Z,label,species,multiplicity,Wyckoff_symbol,occupancy,internal_intersection,...,Temperature,Pressure,CellVolume,FormulaUnitsPerCell,MineralName,MineralGroup,PearsonSymbol,ANXFormula,ABFormula,disorder_set
0,As2 Ba1 Cu1.88,78757,139,2,"{'0': 'Ba1', '1': 'Cu1', '2': 'As1'}","{'0': {'Ba2+': 1.0}, '1': {'Cu2+': 0.940000000...","{'0': 2, '1': 4, '2': 4}","{'0': 'a', '1': 'd', '2': 'e'}","{'0': 1.0, '1': 0.9400000000000001, '2': 1.0}","{'0': False, '1': False, '2': False}",...,293.0,0.101325,199.29,2,NaN,NaN,tI10,AB2X2,A2B3.8C4,"{O, V}"
1,Li0.976 Ni0.7 O1.991 Ti0.3,83283,166,3,"{'0': 'Li1Ni2Ti2', '1': 'O1', '2': 'Ni1Ti1'}","{'0': {'Li1+': 0.976, 'Ni2+': 0.147, 'Ti3+': 0...","{'0': 3, '1': 6, '2': 3}","{'0': 'b', '1': 'c', '2': 'a'}","{'0': 1.174, '1': 0.996, '2': 0.802}","{'0': False, '1': False, '2': False}",...,293.0,0.101325,104.50,3,NaN,NaN,hR4,AB2C3X6,A2.9B3C6,"{SV, S, V}"
2,Be13 Sb1,834,226,8,"{'0': 'Sb1', '1': 'Be2', '2': 'Be1'}","{'0': {'Sb0+': 1.0}, '1': {'Be0+': 1.0}, '2': ...","{'0': 8, '1': 96, '2': 8}","{'0': 'a', '1': 'i', '2': 'b'}","{'0': 1.0, '1': 1.0, '2': 1.0}","{'0': False, '1': False, '2': False}",...,293.0,0.101325,1013.86,8,NaN,NaN,cF112,NO13,AB13,{O}
3,Au1 Ho1 Pb1,58487,216,4,"{'0': 'Au1', '1': 'Ho1', '2': 'Pb1'}","{'0': {'Au0+': 1.0}, '1': {'Ho0+': 1.0}, '2': ...","{'0': 4, '1': 4, '2': 4}","{'0': 'c', '1': 'b', '2': 'a'}","{'0': 1.0, '1': 1.0, '2': 1.0}","{'0': False, '1': False, '2': False}",...,293.0,0.101325,303.19,4,NaN,NaN,cF12,NOP,ABC,{O}
4,C2 Ta1 Zr1,77464,225,2,"{'0': 'Ta1Zr1', '1': 'C1'}","{'0': {'Ta0+': 0.5, 'Zr0+': 0.5}, '1': {'C0+':...","{'0': 4, '1': 4}","{'0': 'b', '1': 'a'}","{'0': 1.0, '1': 1.0}","{'0': False, '1': False}",...,293.0,0.101325,94.45,2,NaN,NaN,cF8,NO,AB,"{O, S}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141649,Al1 F1 Li1 O4 P1,68926,2,4,"{'0': 'O3', '1': 'Li1', '2': 'P1', '3': 'O2', ...","{'0': {'O2-': 1.0}, '1': {'Li1+': 0.5}, '2': {...","{'0': 4, '1': 4, '2': 4, '3': 4, '4': 4, '5': ...","{'0': 'i', '1': 'i', '2': 'i', '3': 'i', '4': ...","{'0': 1.0, '1': 0.5, '2': 1.0, '3': 1.0, '4': ...","{'0': False, '1': False, '2': False, '3': Fals...",...,293.0,0.101325,315.75,4,Montebrasite (F-rich),NaN,aP16,ABCXY4,ABCDE4,"{O, VP}"
141650,Al92 Cd46 N51 O486 Si100,153026,227,1,"{'0': 'Si1Al1', '1': 'Cd1', '2': 'N1', '3': 'O...","{'0': {'Si4+': 0.521, 'Al3+': 0.47900000000000...","{'0': 192, '1': 16, '2': 192, '3': 96, '4': 32...","{'0': 'i', '1': 'c', '2': 'i', '3': 'h', '4': ...","{'0': 1.0, '1': 0.719, '2': 0.133, '3': 1.0, '...","{'0': False, '1': False, '2': False, '3': Fals...",...,294.0,0.101325,15133.37,1,"Zeolite X (Cd-exchanged, dehyd.)",NaN,cF775,A46B51C192X486,A46B51C192D486,"{O, S, V, SVP}"
141651,Sb2 Zn2.91,243919,148,126,"{'0': 'Zn1', '1': 'Zn23', '2': 'Zn2', '3': 'Zn...","{'0': {'Zn2+': 1.0}, '1': {'Zn2+': 0.154}, '2'...","{'0': 6, '1': 18, '2': 6, '3': 18, '4': 18, '5...","{'0': 'c', '1': 'f', '2': 'c', '3': 'f', '4': ...","{'0': 1.0, '1': 0.154, '2': 1.0, '3': 0.576000...","{'0': False, '1': False, '2': False, '3': Fals...",...,293.0,0.101325,14819.99,126,NaN,NaN,hR206,A367X252,A252B367,"{O, VP, V}"
141652,O2880 Si1440,39016,229,1,"{'0': 'O33', '1': 'O18', '2': 'Si16', '3': 'O2...","{'0': {'O2-': 1.0}, '1': {'O2-': 1.0}, '2': {'...","{'0': 48, '1': 96, '2': 96, '3': 48, '4': 48, ...","{'0': 'k', '1': 'l', '2': 'l', '3': 'k', '4': ...","{'0': 1.0, '1': 1.0, '2': 1.0, '3': 1.0, '4': ...","{'0': False, '1': False, '2': False, '3': Fals...",...,293.0,0.101325,90639.86,1,NaN,NaN,cI4320,AX2,AB2,{O}


In [19]:
disorder_sets=[]
for i in range(len(disorder)):
    disorder_sets.append(set(disorder.iloc[i]['orbit_disorder'].values()))
disorder['disorder_set']=disorder_sets

In [36]:
diff_comp=[]
for i in range(len(disorder)):
    comp_el=[]
    for el in Composition(disorder.iloc[i]['formula']).elements:
        comp_el.append(str(el))
    comp_el.sort()
    struct_el=[]
    for spec in disorder.iloc[i]['species'].values():
        for el in spec.keys():
            el=el[:-2]
            struct_el.append(el)
    struct_el=list(set(struct_el))
    struct_el.sort()
    if(comp_el!=struct_el):
        diff_comp.append(i)

In [38]:
disorder.drop(index=diff_comp,inplace=True)
disorder.reset_index(inplace=True,drop=True)
num_el=[]
num_orb=[]
for i in range(len(disorder)):
    num_el.append(len(Composition(disorder.iloc[i]['formula'])))
    num_orb.append(len(disorder.iloc[i]['intersect_orbit_connected']))
disorder['num_el']=num_el
disorder['num_orb']=num_orb
periodic_table=pd.read_csv('data/periodic_table.csv')
elem_list=list(periodic_table['element'].values)

compositions=[]
exc=[]
for i in range(len(disorder)):
    form=disorder.iloc[i]['formula']
    try:
        comp=composition_from_formula(form)
        compositions.append(comp)
    except:
        print('composition error:',i)
        
normalised_compositions=[]
for i,comp in enumerate(compositions):
    try:
        index=len(elem_list)-1
        switch=0
        for el in comp.keys():
            if el not in elem_list:
                switch=1   
        if(switch==0):
            for el in comp.keys():    
                if(elem_list.index(el)<index+1):
                    index=elem_list.index(el)
            devider=float(comp[elem_list[index]])
            for el in comp.keys(): 
                comp[el]=round(float(comp[el])/devider*10000)
            normalised_compositions.append(comp) 
        else:
            print(comp)
            normalised_compositions.append({})
            
    except:
        print(comp)
        normalised_compositions.append({})
        exc.append(i)
merged_compositions=merged_comp(disorder)
disorder['ordered_formula']=merged_compositions
disorder['normalised_composition']=normalised_compositions
disorder=disorder.loc[disorder['normalised_composition']!={}]
disorder.reset_index(drop=True,inplace=True)
com_index=[]
for i in range(len(disorder)):
    orb=disorder.iloc[i]['orbit_disorder'].values()
    if('COM' in orb):
        com_index.append(i)

In [39]:
disorder=disorder.drop(index=com_index)
disorder.reset_index(drop=True,inplace=True)

In [45]:
disorder=disorder.loc[~disorder['entropy'].isna()]
disorder.reset_index(drop=True,inplace=True)

In [47]:
list_of_compounds=list(set(disorder['ordered_formula']))
keep_max=[]
keep_min=[]

for comp in list_of_compounds:
    dx=disorder.loc[disorder['ordered_formula']==comp].copy()
    if(len(dx)>0):
        ind=dx.index.values
        imax=np.argmax(dx['entropy'].values)
        imin=np.argmin(dx['entropy'].values)
        keep_max.append(ind[imax])
        keep_min.append(ind[imin])

            
dbmin=disorder.iloc[keep_min]
dbmax=disorder.iloc[keep_max]

dbmax.reset_index(drop=True,inplace=True)
dbmin.reset_index(drop=True,inplace=True)

dbmax.to_json('data/dbmax.json')
dbmin.to_json('data/dbmin.json')

In [48]:
disorder=dbmax[['formula','disorder_set']]

In [49]:
disorder_type=[]
for dis in disorder['disorder_set'].values:
    if(dis=={'O'}):
        disorder_type.append('O')
    elif(dis=={'S'} or dis=={'S','O'}):
        disorder_type.append('S')
    elif(dis=={'V'} or dis=={'V','O'} or dis=={'VP'} or dis=={'VP','O'} or dis=={'V','VP'} or dis=={'O','V','VP'}):
        disorder_type.append('V')
    else:
        disorder_type.append('M')
disorder['disorder_type']=disorder_type

In [53]:
print('O:',len(disorder.loc[disorder['disorder_type']=='O'])/len(disorder))
print('S:',len(disorder.loc[disorder['disorder_type']=='S'])/len(disorder))
print('V:',len(disorder.loc[disorder['disorder_type']=='V'])/len(disorder))
print('M:',len(disorder.loc[disorder['disorder_type']=='M'])/len(disorder))

O: 0.3809513746539591
S: 0.36148855687749626
V: 0.10660172016652227
M: 0.15095834830202237


In [54]:
g_disorder=pd.DataFrame()
g_disorder['formula']=disorder['formula']
g_disorder['disorder']=disorder_type

In [55]:
g_disorder.to_csv('data/type_disorder.csv')

In [56]:
len(g_disorder)

94642

In [57]:
g_disorder

,formula,disorder
0,Pd4 Sn16,O
1,Ge0.5 Li0.143 Sb0.286 Te1,M
2,F1 Na1 O4 P1 Y1,O
3,Ge1 Nd1 Ru1,O
4,Br4 Cs2 Se8 Tc6,O
...,...,...
94637,Mn2 Nd1 O6 Ta1,O
94638,B7 O12 Y1,O
94639,C5 Al3 Hf2,O
94640,Cs4 Se10 Si4,O


We can encode O=0, S=1, V=2, M=3

In [58]:
dis_type_num=[]
for dis in disorder_type:
    if(dis=='O'):
        dis_type_num.append(0)
    elif(dis=='S'):
        dis_type_num.append(1)
    elif(dis=='V'):
        dis_type_num.append(2)
    elif(dis=='M'):
        dis_type_num.append(3)

In [59]:
g_disorder=pd.DataFrame()
g_disorder['formula']=disorder['formula']
g_disorder['disorder']=dis_type_num

In [60]:
g_disorder.to_csv('data/type_disorder_num.csv')

In [61]:
from sklearn.model_selection import train_test_split

ind=np.linspace(0,len(g_disorder)-1,len(g_disorder))

In [63]:
train_ind,test_ind=train_test_split(ind,test_size=0.2,random_state=12)
train_ind,val_ind=train_test_split(train_ind,test_size=0.1,random_state=12)

In [68]:
train=g_disorder.iloc[train_ind]
train.reset_index(inplace=True,drop=True)
train.to_csv('data/crabnet_data/train.csv')
val=g_disorder.iloc[val_ind]
val.reset_index(inplace=True,drop=True)
val.to_csv('data/crabnet_data/val.csv')
test=g_disorder.iloc[test_ind]
test.reset_index(inplace=True,drop=True)
test.to_csv('data/crabnet_data/test.csv')

In [69]:
val

,formula,disorder
0,Dy0.015 O4 Si1.016 Zr0.985,1
1,Ge4 Nb1 Nd3 Rh4,0
2,Sb1.488 Sn1.612 Ti5,3
3,Ir7 Mg44,0
4,Be0.12 Mo1 Si1.74,3
...,...,...
7567,Eu1 In4,0
7568,O12 P3 Rb2 Ti1 Y1,1
7569,Na2 O5 Si2,0
7570,Br5 Pb2 Tl1,0
